In [1]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_fireworks import ChatFireworks
from langchain_ollama import ChatOllama
from openai.types.beta.threads.runs.code_interpreter_output_image import Image

In [3]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 5-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

llm = ChatOllama(model="qwen3:8b")

In [4]:
generate = prompt | llm

In [5]:
essay = ""
request = HumanMessage(
    content="Write an essay on why the little prince is relevant in modern childhood"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content

<think>
Okay, the user wants an essay on why "The Little Prince" is relevant in modern childhood. Let me start by recalling the main themes of the book. It's about innocence, relationships, and the importance of seeing with the heart. Now, how does that apply to today's kids?

First, I should mention the loss of innocence in today's world. Kids are exposed to technology and social media early, which might make them more cynical. The Little Prince's journey reminds them to stay curious and innocent. Maybe include something about the rose and the baobab, symbolizing love and responsibility.

Next, the importance of relationships. The story shows how the prince learns about friendship and love through his interactions. In a world where kids might feel isolated, the book emphasizes the value of genuine connections. Maybe talk about how the prince's relationships with the fox and the rose teach empathy and care.

Then, the idea of seeing with the heart. The book contrasts logical thinking w

In [6]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [7]:
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content

<think>
Okay, the user wants me to generate a critique and recommendations for their essay on why *The Little Prince* is relevant in modern childhood. Let me start by understanding the essay they wrote. The essay is structured into five paragraphs, each addressing a different theme: innocence, relationships, seeing with the heart, responsibility, and a conclusion. The tone is accessible, and the arguments connect well to modern issues like technology, social media, and emotional development.  

First, I need to evaluate the strengths of the essay. The structure is clear, and the themes are well-chosen. The author effectively ties the book’s messages to contemporary challenges faced by children, such as the loss of innocence and the importance of emotional intelligence. However, there might be areas where the essay could be more detailed or nuanced. For example, the discussion on "seeing with the heart" could delve deeper into specific examples from the text or relate to modern psycholo

In [8]:
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=essay), HumanMessage(content=reflection)]}
):
    print(chunk.content, end="")

<think>
</think>

**Revised Essay: Why *The Little Prince* Remains Relevant in Modern Childhood**  

In today’s fast-paced, technology-driven world, the timeless wisdom of *The Little Prince* continues to resonate deeply with children and parents alike. Written by Antoine de Saint-Exupéry, this beloved novella explores universal themes such as innocence, relationships, and emotional growth, offering profound insights that remain strikingly relevant for modern childhood. As children navigate the complexities of digital communication, social media, and shifting family dynamics, the story’s lessons about love, responsibility, and seeing with the heart provide a much-needed counterbalance to the distractions of contemporary life.  

One of the most enduring messages of *The Little Prince* is its celebration of innocence and the importance of preserving a child’s natural curiosity. In an era where children are often exposed to screens and online interactions from an early age, the story ser

## Define Graph for Automation of above task

In [9]:
from typing import Annotated, List, Sequence
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from typing_extensions import TypedDict

In [10]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

async def generation_node(state: State) -> State:
    return {"messages": [await generate.ainvoke(state["messages"])]}

async def reflection_node(state: State) -> State:
    # Other messages we need to adjust
    cls_map = {"ai" : HumanMessage, "human" : AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [state["messages"][0]] + [
        cls_map[msg.type](content=msg.content) for msg in state["messages"][1:]
    ]
    res = await reflect.ainvoke(translated)
    # We treat the output of this as human feedback for the generator
    return {"messages": [HumanMessage(content=res.content)]}

In [11]:
def should_continue(state: State):
    if len(state["messages"]) > 6:
        # End after 3 iterations
        return END
    return "reflect"

In [12]:
builder = StateGraph(State)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_edge(START, "generate")
builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [15]:
# from IPython.display import display, Image
# display(Image(graph.get_graph().draw_mermaid_png()))

In [16]:
config = {"configurable": {"thread_id": "1"}}

In [17]:
async for event in graph.astream(
    {
        "messages": [
            HumanMessage(
                content="Generate an essay on the topicality of The Little Prince and its message in modern life"
            )
        ],
    },
    config,
):
    print(event)
    print("---")

{'generate': {'messages': [AIMessage(content='<think>\nOkay, the user wants an essay on the topicality of The Little Prince and its message in modern life. Let me start by recalling the main themes of the book. The story is about a young prince who travels from planet to planet, learning about love, loss, and the essence of things. The key messages include the importance of relationships, the value of simplicity, and the dangers of materialism.\n\nNow, how does this relate to modern life? The user is probably looking for connections to today\'s issues like technology overuse, social media\'s impact, and the fast pace of life. Maybe I should discuss how the book\'s themes about valuing human connections over material gains are still relevant. Also, the idea of seeing with the heart versus the eyes is a metaphor for not being distracted by superficial things.\n\nI need to structure the essay into five paragraphs. The introduction should present the topic and thesis. The next paragraphs c

In [18]:
state = graph.get_state(config)

In [19]:
ChatPromptTemplate.from_messages(state.values["messages"]).pretty_print()

================================ Human Message =================================

Generate an essay on the topicality of The Little Prince and its message in modern life

================================== Ai Message ==================================

<think>
Okay, the user wants an essay on the topicality of The Little Prince and its message in modern life. Let me start by recalling the main themes of the book. The story is about a young prince who travels from planet to planet, learning about love, loss, and the essence of things. The key messages include the importance of relationships, the value of simplicity, and the dangers of materialism.

Now, how does this relate to modern life? The user is probably looking for connections to today's issues like technology overuse, social media's impact, and the fast pace of life. Maybe I should discuss how the book's themes about valuing human connections over material gains are still relevant. Also, the idea of seeing with the heart versus 